In [1]:
import dimacs

class Node:
    def __init__(self, idx):
        self.idx = idx
        self.out = set()              # zbiór sąsiadów

    def connect_to(self, v):
        self.out.add(v)
                
        
def loadGraph(fileName):
    fileContent = dimacs.loadWeightedGraph(fileName)
    G = [Node(i) for i in range(fileContent[0])] 
    for (u, v, _) in fileContent[1]:
        G[u-1].connect_to(v-1)
        G[v-1].connect_to(u-1)
    return G

In [36]:
def LexBFS(visited, graph):
    vertices = []
    
    vertices = [x.idx for x in graph]
    parents = [-1 for x in graph]
    predecessors = [set() for x in graph]
    listOfSets = [set(vertices)]
    
    
    ordering = []
    
    
    while listOfSets:
        while listOfSets and len(listOfSets[-1]) == 0 :
            listOfSets.pop()
            
        if len(listOfSets) == 0 : break
            
        vertex = listOfSets[-1].pop()
        
        ordering.append(vertex)
        newList = []
        for x in listOfSets:
            notNeighbours  = x - graph[vertex].out
            Neighbours = x & graph[vertex].out
            for y in Neighbours:
                parents[y] = vertex
                predecessors[y].add(vertex)
            newList.append(notNeighbours)
            newList.append(Neighbours)
        listOfSets = newList.copy()
        
    return ordering,parents, predecessors

In [106]:
graph = loadGraph('chordal\interval-rnd6')
ordering,parents,predecessors = LexBFS(5,graph)
print(ordering)
print(parents)
print(predecessors)

[0, 1, 2, 4, 3, 5]
[-1, 0, 1, 2, 1, 3]
[set(), {0}, {0, 1}, {1, 2}, {0, 1}, {1, 2, 3}]


In [107]:
class SpanningNode:
    def __init__(self,vertexId):
        self.id = vertexId
        self.childList = []

class SpanningTree:
    def __init__(self,root):
        self.root = root
        
    def __findNodeRecur__(self,vertex, parent):
        if (vertex.id == parent):
            return vertex
        if vertex.childList == []:
            return None
        for x in vertex.childList:
            node = self.__findNodeRecur__(x,parent)
            if not node is None:
                return node
        
    def findNode(self,id):
        return self.__findNodeRecur__(self.root, id)
    

    def insertNode(self,id,node):
        child = SpanningNode(id)
        node.childList.append(child)
       


In [108]:
order = ordering.copy()
tree = SpanningTree(SpanningNode(order.pop(0)))

for x in order:
    tree.insertNode(x, tree.findNode(parents[x]))
    


In [109]:
Cliques = [[[],-1] for x in graph]
cliqueofVertex = [x for x in range(len(graph))]

def buildCliques(id, cliques, predecessors, cliqueofVertex):
    if (list(predecessors[id]) == cliques[cliqueofVertex[parents[id]]][0]):
        cliques[cliqueofVertex[parents[id]]][0].append(id)
        cliqueofVertex[id] = cliqueofVertex[parents[id]]
    else :
        Cliques[cliqueofVertex[id]][0] = list(predecessors[id]) + [id]
        Cliques[cliqueofVertex[id]][1] = parents[id]
        

In [110]:
def cliquesPreorder(node, cliques,predecessors,cliqueOfVertex,ordering):
    if node == tree.root:
        cliques[cliqueOfVertex[node.id]][0].append(node.id)
    buildCliques(node.id, cliques, predecessors, cliqueOfVertex)
    for x in ordering:
        for y in node.childList:
            if (x == y.id):
                cliquesPreorder(y, cliques,predecessors,cliqueOfVertex,ordering)
        

In [111]:
cliquesPreorder(tree.root, Cliques, predecessors, cliqueofVertex, ordering)

print(Cliques)
print(cliqueofVertex)


[[[0], -1], [[], -1], [[], -1], [[1, 2, 3, 5], 2], [[0, 1, 4], 1], [[0, 1, 2], -1]]
[5, 5, 5, 3, 4, 3]
